In [2]:
import json
import os
import requests
from bs4 import BeautifulSoup
import time

# Directory containing JSON files
json_dir = 'NVD_CVE_7-23'

# Function to extract URLs tagged as "exploit" from a JSON file
def extract_exploit_urls_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        urls = []
        references = data.get('cve', {}).get('references', [])
        for ref in references:
            if 'tags' in ref and 'Exploit' in ref['tags']:
                urls.append(ref['url'])
        return urls


# Function to scrape content from a URL with a delay and save titles, headings, and body text to a file
def scrape_url(url, output_file, delay=1):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.title.string if soup.title else 'No title found'
        print(f"Title: {title}")
        
        headings = [heading.get_text() for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
        body = soup.get_text()
        
        with open(output_file, 'a') as f:
            f.write(f"Title: {title}\n")
            f.write("Headings:\n")
            for heading in headings:
                f.write(f"{heading}\n")
            f.write("Body:\n")
            f.write(f"{body}\n\n")
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")
    finally:
        time.sleep(delay)  # Delay between requests

# Collect and process each JSON file
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        json_file = os.path.join(json_dir, filename)
        exploit_urls = extract_exploit_urls_from_json(json_file)
        if exploit_urls:
            output_file = os.path.join(json_dir, f"{os.path.splitext(filename)[0]}.txt")
            print(f"Extracted {len(exploit_urls)} exploit URLs from {filename}.")
            
            # Ensure the output file is empty before writing
            with open(output_file, 'w') as f:
                f.write('')
            
            # Scrape content from all extracted "exploit" URLs with a delay and save titles, headings, and body text to the file
            for url in exploit_urls:
                scrape_url(url, output_file)


Extracted 1 exploit URLs from CVE-2024-3171.json.


NameError: name 'output_file_txt' is not defined